# Demostración de uso

El paquete se llama muTel (de Muon Telescope) y en un principio englobaría toda la parte de DAQ y DQM. En este caso, nos centraremos en el DQM.

## MuData

MuData es la estructura básica de datos que vamos a manejar, es un envoltorio bonito de un pd.DataFrame que nos va a permitir aplicarle métodos de forma más cómoda y compacta.

In [3]:
import importlib.util
import sys

loc = r"F:\Clase\03 - Máster\TFM\Programa\muTel\src\muTel\__init__.py"
# loc = "muTel_work/muTel_v02/muTel/src/muTel/__init__.py"
# loc = "/afs/ciemat.es/user/m/martialc/muTel/src/muTel/__init__.py"
spec = importlib.util.spec_from_file_location("muTel", loc)
foo = importlib.util.module_from_spec(spec)
sys.modules["muTel"] = foo
spec.loader.exec_module(foo)

In [4]:
from muTel.dqm.classes.MuData import MuData
from muTel.utils.config import load_cfg
from muTel.dqm.classes.Filters import Drop, DropTrigger, SLStudy, TimeFrame

In [5]:
data = MuData(588)

Al crear un objeto con la clase MuData, tendremos que indicar el número de la Run, así como la SL (en caso de que estén separadas) y el número de hits (en caso de que esté separado). Es decir, usa el mismo paradigma para leer los archivos que el que se usa para nombrarlos.

### Suma

Los objetos MuData se pueden sumar, dando lugar a un nuevo conjunto de datos formados por los dos de la suma:

[Esto es todavía bastante experimental]

(Para ejecutarla, hay que cambiar el tipo de celda a Python. Está en raw para que no se ejecute al ejecutar toda la libreta, ya que tarda un ratillo y no sirve para mucho en ese ejemplo.)

### Métodos

#### display_event


He hecho una función que te permite pintar una tabla con un gráfico de barras para tener una mejor idea de la distribución de tiempos. El máximo y el mínimo están normalizados al filtro de tiempos (TimeFrame). Toma de valor del evento que se quiere representar:

In [ ]:
data.display_event(51239)

#### calculate_T0

Permite calcular el valor de T0 para cada una de las SL con su incertidumbre y lo guarda en el objeto como self.T0:

In [13]:
data_ex = data.copy()
data_ex.add_filter(load_cfg(Drop))
data_ex.add_filter(load_cfg(DropTrigger))
data_ex.add_filter(load_cfg(TimeFrame))
data_ex.add_filter(load_cfg(SLStudy,config_name='01_4hits_in'))


In [18]:
data_ex.calculate_T0()

,T0,dT0
sl,,
1,709.344070,5.496488
2,709.176662,5.614921
3,709.962084,3.136651
4,711.089938,2.728640


## Próximamente

Mi idea es que estos objetos te permitan también calcular T0, los meantimers, y los distintos gráficos con métodos que tengan dentro, y así tener todo aglutinado en un mismo lugar.

## Filter

Filter es la clase que engloba todos los filtros. Por sí misma, no hace nada, son sus distintos hijas las que tendrán distintas funciones.

Los filtros tienen conjuntos de valores por defecto que se guardan en un archivo JSON para poder ser cargados con facilidad. Esto se hace mediante la función "load_cfg". Se le da como argumento el filtro y, de forma opcional, el nombre de la configuración (por defecto, el valor es "default").

Este archivo JSON se puede encontrar en "config/filters.json" dentro del directorio del paquete.

### Drop

Este filtro sirve para quitar columnas del conjunto de datos. En este caso, en vez de indicar las columnas que se van a despreciar, se indican las que se quieren mantener.

Estos son los datos sin filtrar las columnas:

In [ ]:
display(data)

Y estos son los datos después de aplicar el filtro:

In [ ]:
data.add_filter(load_cfg(Drop))
display(data)

Con los valores por defecto de Drop se conservan las columnas:
- EventNr
- GEO
- hit
- channel
- sl
- layer
- cell
- DriftTime

### DropTrigger

Con la ayuda de este filtro, eliminamos las señales provenientes del canal del trigger:

In [ ]:
data.add_filter(load_cfg(DropTrigger))
display(data)

### TimeFrame

Con este filtro podemos seleccionar una ventana de tiempo en la que filtrar los datos. Los valores por defecto son tmin = 600 y tmax = 1200.

In [ ]:
data.add_filter(load_cfg(TimeFrame))
display(data)

### SLStudy

Este filtro es el que se encargará de seleccionar los eventos que tengan al menos 4 hits en una de las SL. Hay dos modos de funcionamiento:
- <u>Inclusivo</u>: 4 hits en **al menos 1** de las capas.
- <u>Exclusivo</u>: 4 hits en **todas** las capas.

Se le indica la SL que tiene que mirar y el modo de funcionamiento.

#### Inclusivo

In [ ]:
muon_sl_1_in= data.copy()
muon_sl_1_in.add_filter(load_cfg(SLStudy,config_name='01_4hits_in'))

In [ ]:
print('Número de final de entradas:',len(muon_sl_1_in))
display(muon_sl_1_in)

#### Exclusivo:

In [ ]:
muon_ex= data.copy()
muon_ex.add_filter(load_cfg(SLStudy,config_name='4hits_ex'))

In [ ]:
print('Número de final de entradas:',len(muon_ex))
display(muon_ex)

Comparando los resultados antes y después de ejecutar el filtro de 4hits:

In [ ]:
test_cfg  =['01_4hits_in','02_4hits_in','03_4hits_in','04_4hits_in','4hits_ex']

for i, cfg in enumerate(test_cfg):
    # print(f'cfg: {cfg}')
    data_test = data.copy()
    data_test.add_filter(load_cfg(SLStudy,config_name=cfg))
    print(f'Eficiencia a 4 hits de {f"SL {i+1}" if i < 4 else "todas"}: {data_test.NEvents()/data.NEvents()*100:.1f} %')